#  Incident Root Cause Analysis 

Incident Reports in ITOps usually states the symptoms. Identifying the root cause of the symptom quickly is a key determinant to reducing resolution times and improving user satisfaction.

# 06.02. Preprocessing Incident Data

### Loading the Dataset

In [2]:
import pandas as pd
import os
import tensorflow as tf

#Load the data file into a Pandas Dataframe
symptom_data = pd.read_csv("/Users/disastershubz/Downloads/Ex_Files_Deep_Learning_Getting_Started/Exercise Files/root_cause_analysis.csv")

#Explore the data loaded
print(symptom_data.dtypes)
symptom_data.head()

ID                   int64
CPU_LOAD             int64
MEMORY_LEAK_LOAD     int64
DELAY                int64
ERROR_1000           int64
ERROR_1001           int64
ERROR_1002           int64
ERROR_1003           int64
ROOT_CAUSE          object
dtype: object


,ID,CPU_LOAD,MEMORY_LEAK_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003,ROOT_CAUSE
0,1,0,0,0,0,1,0,1,MEMORY_LEAK
1,2,0,0,0,0,0,0,1,MEMORY_LEAK
2,3,0,1,1,0,0,1,1,MEMORY_LEAK
3,4,0,1,0,1,1,0,1,MEMORY_LEAK
4,5,1,1,0,1,0,1,0,NETWORK_DELAY


### Convert  data

Input data needs to be converted to formats that can be consumed by ML algorithms

In [5]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

label_encoder = preprocessing.LabelEncoder()
symptom_data['ROOT_CAUSE'] = label_encoder.fit_transform(
                                symptom_data['ROOT_CAUSE'])

#Convert Pandas DataFrame to a numpy vector
np_symptom = symptom_data.to_numpy().astype(float)

#Extract the feature variables (X)
X_data = np_symptom[:,1:8]

#Extract the target variable (Y), conver to one-hot-encodign
Y_data=np_symptom[:,8]
Y_data = tf.keras.utils.to_categorical(Y_data,3)

#Split training and test data
X_train,X_test,Y_train,Y_test = train_test_split( X_data, Y_data, test_size=0.10)

print("Shape of feature variables :", X_train.shape)
print("Shape of target variable :",Y_train.shape)

Shape of feature variables : (900, 7)
Shape of target variable : (900, 3)


## 06.03. Building and evaluating the model

In [8]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2

#Setup Training Parameters
EPOCHS=20
BATCH_SIZE=64
VERBOSE=1
OUTPUT_CLASSES=len(label_encoder.classes_)
N_HIDDEN=128
VALIDATION_SPLIT=0.2

#Create a Keras sequential model
model = tf.keras.models.Sequential()
#Add a Dense Layer
model.add(keras.layers.Dense(N_HIDDEN,
                             input_shape=(7,),
                              name='Dense-Layer-1',
                              activation='relu'))

#Add a second dense layer
model.add(keras.layers.Dense(N_HIDDEN,
                              name='Dense-Layer-2',
                              activation='relu'))

#Add a softmax layer for categorial prediction
model.add(keras.layers.Dense(OUTPUT_CLASSES,
                             name='Final',
                             activation='softmax'))

#Compile the model
model.compile(
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()

#Build the model
model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)


#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset :\n------------------------------------")
model.evaluate(X_test,Y_test)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Dense-Layer-1 (Dense)           │ (None, 128)            │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense-Layer-2 (Dense)           │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Final (Dense)                   │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,923 (70.01 KB)

 Trainable params: 17,923 (70.01 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5210 - loss: 1.0395 - val_accuracy: 0.7778 - val_loss: 0.8170
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - accuracy: 0.7963 - loss: 0.7597 - val_accuracy: 0.8111 - val_loss: 0.6370
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - accuracy: 0.8156 - loss: 0.6003 - val_accuracy: 0.8333 - val_loss: 0.5465
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.7882 - loss: 0.5480 - val_accuracy: 0.8222 - val_loss: 0.4960
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 903us/step - accuracy: 0.8375 - loss: 0.4782 - val_accuracy: 0.8222 - val_loss: 0.4685
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - accuracy: 0.8292 - loss: 0.4619 - val_accuracy: 0.8389 - val_loss: 0.4653
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step - accuracy: 0.8290 - loss: 0.4599 - val_accuracy: 0.8333 - val_loss: 0.4466
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step - accuracy: 0.8267 - loss: 0.4798 - val_accuracy: 0.83

[0.4027225375175476, 0.8600000143051147]

## 06.04. Predicting Root Causes

In [13]:
import numpy as np

# Define individual flags
CPU_LOAD = 1
MEMORY_LOAD = 0
DELAY = 0
ERROR_1000 = 0
ERROR_1001 = 1
ERROR_1002 = 1
ERROR_1003 = 0

# Ensure the input is a NumPy array
input_data = np.array([[CPU_LOAD, MEMORY_LOAD, DELAY, ERROR_1000, ERROR_1001, ERROR_1002, ERROR_1003]])

# Make a prediction
prediction = np.argmax(model.predict(input_data), axis=1)

# Decode the prediction using the label encoder
print(label_encoder.inverse_transform(prediction))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
['DATABASE_ISSUE']


In [17]:
#import numpy as np

# Define the batch input
batch_input = np.array([
    [1, 0, 0, 0, 1, 1, 0],
    [0, 1, 1, 1, 0, 0, 0],
    [1, 1, 0, 1, 1, 0, 1],
    [0, 0, 0, 0, 0, 1, 0],
    [1, 0, 1, 0, 1, 1, 1]
])

# Predict for the batch
predictions = model.predict(batch_input)

# Get the class with the highest probability for each prediction
predicted_classes = np.argmax(predictions, axis=1)

# Convert the predictions back to the original labels
decoded_labels = label_encoder.inverse_transform(predicted_classes)

# Print the decoded labels
print(decoded_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
['DATABASE_ISSUE' 'NETWORK_DELAY' 'MEMORY_LEAK' 'DATABASE_ISSUE'
 'DATABASE_ISSUE']
